In [1]:
# импортируем библиотеки pandas, numpy
import pandas as pd
import numpy as np
# импортируем модуль os и функцию train_test_split()
import os
from sklearn.model_selection import train_test_split
# импортируем класс StandardScaler, 
# выполняющий стандартизацию
from sklearn.preprocessing import StandardScaler
# импортируем класс LogisticRegression
from sklearn.linear_model import LogisticRegression

In [2]:
# взглянем на наш рабочий каталог
os.getcwd()

'/Users/artemgruzdev/Documents/GitHub/Data Preprocessing in Python'

In [3]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/StateFarm.csv', sep=';') 

In [4]:
# смотрим данные
data.head(3)

,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Response
0,18975.456110,65999,237,1,14,0,6,0
1,4715.321344,0,65,19,56,0,3,0
2,5018.885233,54500,63,28,17,0,6,0


In [5]:
# разбиваем данные на обучающие и тестовые: получаем обучающий
# массив признаков, тестовый массив признаков, обучающий массив
# меток, тестовый массив меток
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('Response', axis=1), 
    data['Response'], 
    test_size=0.3,
    stratify=data['Response'],
    random_state=42)

In [6]:
# создаем экземпляр класса StandardScaler
standardscaler = StandardScaler()

In [7]:
# обучаем модель стандартизации, т.е. по каждому признаку 
# в обучающем массиве признаков вычисляем
# среднее значение признака и стандартное 
# отклонение признака для трансформации
standardscaler.fit(X_train);

In [8]:
# применяем модель стандартизации к обучающему массиву признаков: из исходного 
# значения признака вычитаем среднее значение признака, вычисленное 
# по ОБУЧАЮЩЕМУ массиву признаков, и результат делим на стандартное 
#отклонение признака, вычисленное по ОБУЧАЮЩЕМУ массиву признаков 
X_train_standardscaled = standardscaler.transform(X_train)

# применяем модель стандартизации к тестовому массиву признаков: из исходного 
# значения признака вычитаем среднее значение признака, вычисленное 
# по ОБУЧАЮЩЕМУ массиву признаков, и результат делим на стандартное 
# отклонение признака, вычисленное по ОБУЧАЮЩЕМУ массиву признаков 
X_test_standardscaled = standardscaler.transform(X_test)

In [9]:
# создаем экземпляр класса LogisticRegression
logreg = LogisticRegression(solver='lbfgs', max_iter=200)
# обучаем модель логистической регрессии, т.е. 
# находим параметры - регрессионные коэффициенты
logreg.fit(X_train_standardscaled, y_train)
# оцениваем качество модели на обучающих данных
print("Правильность на обучающей выборке: {:.3f}".format(
    logreg.score(X_train_standardscaled, y_train)))
# оцениваем качество модели на тестовых данных
print("Правильность на тестовой выборке: {:.3f}".format(
    logreg.score(X_test_standardscaled, y_test)))

Правильность на обучающей выборке: 0.900
Правильность на тестовой выборке: 0.900


In [10]:
# применим для печати f-строки
train_score = logreg.score(X_train_standardscaled, y_train)
test_score = logreg.score(X_test_standardscaled, y_test)
print(f"Правильность на обучающей выборке: {train_score:.3f}")
print(f"Правильность на тестовой выборке: {test_score:.3f}")

Правильность на обучающей выборке: 0.900
Правильность на тестовой выборке: 0.900


In [11]:
# применим для печати оператор % 
print("Правильность на обучающей выборке: %.3f" % train_score)
print("Правильность на тестовой выборке: %.3f" % test_score)

Правильность на обучающей выборке: 0.900
Правильность на тестовой выборке: 0.900


In [12]:
# вычисляем спрогнозированные значения зависимой переменной
# для тестового массива признаков
logreg_predvalues = logreg.predict(X_test_standardscaled)
logreg_predvalues[:5]

array([0, 0, 0, 0, 0])

In [13]:
# вычисляем вероятности классов зависимой переменной
# для тестового массива признаков
logreg_probabilities = logreg.predict_proba(
    X_test_standardscaled)
logreg_probabilities[:5]

array([[0.91027855, 0.08972145],
       [0.89847518, 0.10152482],
       [0.88300257, 0.11699743],
       [0.90234211, 0.09765789],
       [0.92554507, 0.07445493]])

In [14]:
# извлекаем константу
intercept = np.round(logreg.intercept_[0], 3)
intercept

-2.205

In [15]:
# извлекаем коэффициенты
coef = np.round(logreg.coef_, 3)
coef

array([[-0.022,  0.042,  0.079, -0.056, -0.004, -0.017, -0.109]])

In [16]:
# печатаем название "Константа"    
print("Константа:", intercept)
# печатаем название "Регрессионные коэффициенты"
print("Регрессионные коэффициенты:")
# для удобства сопоставим каждому названию 
# предиктора соответствующий коэффициент
for c, feature in zip(coef[0], X_train.columns):
    print(feature, c)

Константа: -2.205
Регрессионные коэффициенты:
Customer Lifetime Value -0.022
Income 0.042
Monthly Premium Auto 0.079
Months Since Last Claim -0.056
Months Since Policy Inception -0.004
Number of Open Complaints -0.017
Number of Policies -0.109
